In [1]:
%autoawait
%reset

import logging
import pyuavcan  # READ THE LIBRARY DOCS AT http://pyuavcan.readthedocs.io

# In this interactive scenario there is no point generating DSDL packages programmatically because
# it is easier to invoke the DSDL code generator from the command line instead.
# For the purposes of this demo, the following command will do the job (in the same directory):
#    yakut compile dsdl/example https://github.com/UAVCAN/public_regulated_data_types/archive/master.zip

import uavcan   # Generated package for the DSDL namespace "uavcan".
import example  # Generated package for the DSDL namespace "example".
from pyuavcan.application import make_node, NodeInfo, register

node = make_node(
    NodeInfo(),
    schema={
        "uavcan.serial.port": register.Value(string=register.String("loop://")),
        "uavcan.node.id":  register.Value(natural16=register.Natural16([250])),
        "uavcan.pub.a.id": register.Value(natural16=register.Natural16([1234])),
        "uavcan.sub.a.id": register.Value(natural16=register.Natural16([1234])),
    },
)
node.start()

IPython autoawait is `on`, and set to use `asyncio`
Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
pub_a = node.make_publisher(example.MyMessage_0_1, 'a')
sub_a = node.make_subscriber(example.MyMessage_0_1, 'a')

In [3]:
import errno
import uavcan.file

async def file_read_handler(
    request: uavcan.file.Read_1_0.Request,
    meta: pyuavcan.presentation.ServiceRequestMetadata,
) -> uavcan.file.Read_1_0.Response:
    print('File read request', request, 'with metadata', meta)
    try:
        with open(request.path.path.tobytes().decode(), 'r') as f:
            f.seek(request.offset)
            data = f.read(256)
        return uavcan.file.Read_1_0.Response(data=data)
    except OSError as ex:
        error_value = {
            errno.EACCES:   uavcan.file.Error_1_0.ACCESS_DENIED,
            errno.E2BIG:    uavcan.file.Error_1_0.FILE_TOO_LARGE,
            errno.EINVAL:   uavcan.file.Error_1_0.INVALID_VALUE,
            errno.EIO:      uavcan.file.Error_1_0.IO_ERROR,
            errno.EISDIR:   uavcan.file.Error_1_0.IS_DIRECTORY,
            errno.ENOENT:   uavcan.file.Error_1_0.NOT_FOUND,
            errno.ENOTSUP:  uavcan.file.Error_1_0.NOT_SUPPORTED,
            errno.ENOSPC:   uavcan.file.Error_1_0.OUT_OF_SPACE,
        }.get(ex.errno, uavcan.file.Error_1_0.UNKNOWN_ERROR)
        return uavcan.file.Read_1_0.Response(error=error_value)

server_file_read = node.get_server(uavcan.file.Read_1_0)
server_file_read.serve_in_background(file_read_handler)

In [4]:
await pub_a.publish(example.MyMessage_0_1(value=123.456))

message, metadata = await sub_a.receive_for(1.0)
print(message)
print(metadata)

example.MyMessage.0.1(value=123.456)
TransferFrom(2021-02-22T13:56:24.646945/1547360.971316, priority=NOMINAL, transfer_id=0, fragmented_payload=[8B], source_node_id=250)


In [5]:
client_file_read = node.make_client(uavcan.file.Read_1_0, server_node_id=250)

file_read_request = uavcan.file.Read_1_0.Request(offset=2, path=uavcan.file.Path_1_0('README.md'))
response, metadata = await client_file_read.call(file_read_request)

assert isinstance(response, uavcan.file.Read_1_0.Response)

first_line = response.data.tobytes().decode().split('\n')[0]
print('First line of the returned text:', repr(first_line))
print('Response metadata:', metadata)

File read request uavcan.file.Read.Request.1.0(offset=2, path=uavcan.file.Path.1.0(path='README.md')) with metadata ServiceRequestMetadata(2021-02-22T13:56:27.786130/1547364.110501, priority=NOMINAL, transfer_id=0, client_node_id=250)
First line of the returned text: 'Jupyter PyUAVCAN demo'
Response metadata: TransferFrom(2021-02-22T13:56:27.789650/1547364.114020, priority=NOMINAL, transfer_id=0, fragmented_payload=[260B], source_node_id=250)


In [6]:
import json

info_client = node.make_client(uavcan.node.GetInfo_1_0, server_node_id=250)
response, metadata = await info_client.call(uavcan.node.GetInfo_1_0.Request())

print('Node info as JSON:', json.dumps(pyuavcan.dsdl.to_builtin(response), indent=4))

Node info as JSON: {
    "protocol_version": {
        "major": 1,
        "minor": 0
    },
    "hardware_version": {
        "major": 0,
        "minor": 0
    },
    "software_version": {
        "major": 0,
        "minor": 0
    },
    "software_vcs_revision_id": 0,
    "unique_id": [
        77,
        128,
        170,
        31,
        11,
        78,
        243,
        204,
        70,
        113,
        96,
        35,
        231,
        233,
        242,
        46
    ],
    "name": "anonymous.4d80aa1f0b4ef3cc46716023e7e9f22e",
    "software_image_crc": [],
    "certificate_of_authenticity": ""
}
